In [20]:
import pandas as pd

trans_survey_filepath = "../US_trans_survey_2015_v_2022/percentages_from_US_trans_survey2015_vs_2022.csv"
with open(trans_survey_filepath, "r") as trans_survey_file:
    og_trans_survey_df = pd.read_csv(filepath_or_buffer=trans_survey_file, index_col=0)

directions_df = og_trans_survey_df.copy()
directions_df.insert(loc=1, column="non_crossdressers_total", value=100-directions_df["crossdressers"])
directions_df.insert(loc=1, column="transmascs_%", value=round((directions_df["transmasc_total"]/directions_df["non_crossdressers_total"]*100), 2))
directions_df.insert(loc=1, column="transfemmes_%", value=round((directions_df["transfemme_total"]/directions_df["non_crossdressers_total"]*100), 2))
directions_df = directions_df.get(["transfemmes_%","transmascs_%"])
directions_df = directions_df.transpose()
print(directions_df)

trans_survey_df = og_trans_survey_df.get([
    "FtM",
    "MtF",
    "FtX",
    "MtX",
    "crossdressers"
])
trans_survey_df = trans_survey_df.transpose()
print(trans_survey_df)

nb_df = og_trans_survey_df.get(["NB_percentage_of_transmascs","NB_percentage_of_transfemmes"])
nb_df.insert(loc=1, column="FTM_%_of_transmascs", value=100-nb_df["NB_percentage_of_transmascs"])
nb_df.insert(loc=3, column="MTF_%_of_transfemmes", value=100-nb_df["NB_percentage_of_transfemmes"])
nb_df = nb_df.rename(
    columns={
        "NB_percentage_of_transmascs": "NB_%_of_transmascs",
        "NB_percentage_of_transfemmes": "NB_%_of_transfemmes"
    }
).transpose()
print(nb_df)


Year            2015   2022
transfemmes_%  41.24  43.88
transmascs_%   58.76  56.12
Year           2015  2022
FtM              29    25
MtF              33    35
FtX              28    30
MtX               7     8
crossdressers     3     2
Year                  2015  2022
NB_%_of_transmascs    49.1  54.5
FTM_%_of_transmascs   50.9  45.5
NB_%_of_transfemmes   17.5  18.6
MTF_%_of_transfemmes  82.5  81.4


In [30]:
# total gender distribution per year

import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels =["Trans Men (FtM)", "Trans Women (MtF)", "AFAB NBs (FtX)", "AMAB NBs (MtX)", "Crossdressers"]

total_fig = make_subplots(rows=1, cols=2, specs=[[
    {'type':'domain'}, {'type':'domain'}
]])

total_fig.add_trace(go.Pie(
    labels=labels, 
    values=trans_survey_df[2015], 
    hole=0.4, 
    title="2015", 
    sort=False,
    titlefont={
        "size":20
    }
), 1, 1)
total_fig.add_trace(go.Pie(
    labels=labels, 
    values=trans_survey_df[2022], 
    hole=0.4, 
    title="2022", 
    sort=False,
    titlefont={
        "size":20
    }
), 1, 2)

colours = ['hotpink', 'aquamarine', "purple", 'lightseagreen', "darkslateblue"] 
# colours = ["lightpink", "lightskyblue", "plum", "lightgreen", "lemonchiffon"]

total_fig.update_traces(hoverinfo="label+percent", # to update what is shown on hover info!
                  textinfo='label+percent',
                  marker=dict(colors=colours, line=dict(color='#000000', width=2))) # changing colours & adding outline o.o

total_fig.update_layout(title="Gender distribution of participants", showlegend=False)


total_fig.show()


In [6]:
# transfemmes vs transmascs ratio per year

direction_labels = ["Transfemmes (AMAB)", "Transmascs (AFAB)"]

direction_fig = make_subplots(rows=1, cols=2, specs=[[
    {'type':'domain'}, {'type':'domain'}
]])

direction_fig.add_trace(go.Pie(
    labels=direction_labels, 
    values=directions_df[2015], 
    hole=0.4, 
    title="2015", 
    titlefont={
        "size":20
    }
), 1, 1)
direction_fig.add_trace(go.Pie(
    labels=direction_labels, 
    values=directions_df[2022], 
    hole=0.4, 
    title="2022", 
    titlefont={
        "size":20
    }
), 1, 2)

colours = ['aquamarine', 'hotpink'] 

direction_fig.update_traces(hoverinfo="label+percent", # to update what is shown on hover info!
                  textinfo='percent',
                  marker=dict(colors=colours, line=dict(color='#000000', width=2))) # changing colours & adding outline o.o

direction_fig.update_layout(title="Transfemme/transmasc distribution")

direction_fig.show()

In [28]:
# gender distribution changes over 7 years

genders = labels[:-1]

change_fig = go.Figure()
change_fig.add_trace(go.Bar(
    x=genders,
    y=trans_survey_df[2015],
    name='2015',
    marker_color='darkmagenta',
    text="2015"
))
change_fig.add_trace(go.Bar(
    x=genders,
    y=trans_survey_df[2022],
    name='2022',
    marker_color='darkorchid',
    text="2022"
))

change_fig.update_layout(barmode='group', showlegend=False, title="Gender distribution changes over 7 years of trans & nb awareness raising")
change_fig.show()


In [31]:
# nbs vs binaries per direction!

# Year                  2015  2022
# NB_%_of_transmascs    49.1  54.5
# FTM_%_of_transmascs   50.9  45.5
# NB_%_of_transfemmes   17.5  18.6
# MTF_%_of_transfemmes  82.5  81.4

fig=go.Figure()

nb_labels = ["NB", "binary"]

for index in [0,1,2,3]:
    index_colours = {
        0: "purple", # afab nbs
        1: "hotpink", # trans guys
        2: "lightseagreen", # nb transfemmes
        3: "aquamarine", # trans women
    }

    if index%2==0:
        label = nb_labels[0]
    else: label = nb_labels[1]

    if index <= 1:
        x_name = "transmascs"
    else: x_name = "transfemmes"

    fig.add_trace(go.Bar(
            x=["2015" + " " + x_name],
            y=[list(nb_df[2015])[index]],
            text=label,
            marker_color=index_colours[index],
            
        )
    )
    fig.add_trace(go.Bar(
            x=["2022" + " " + x_name],
            y=[list(nb_df[2022])[index]],
            text=label,
            marker_color=index_colours[index],
        )
    )

fig.update_layout(
    barmode='stack', 
    showlegend=False, 
    title="Percentual nonbinary distribution by direction"
)

fig.show()